# Phase 2.2 — LoRA Fine-Tuning (Qwen 2.5-7B → Credit Signal Extraction)

**What this notebook does:**
1. Loads Qwen 2.5-7B-Instruct in 4-bit QLoRA configuration
2. Applies LoRA adapters (r=16, targeting attention + MLP layers)
3. Fine-tunes on 9,591 training examples using TRL's SFTTrainer
4. Evaluates on validation set (2,247 examples, 2022-01 to 2023-06)
5. Evaluates on test set (2,133 examples, 2023-07 to 2024-12)
6. Evaluates on entity holdout (3,303 examples: DHFL, Reliance Capital, Cholamandalam)

**Why LoRA instead of full fine-tuning:**
Full fine-tuning updates all 7.6B parameters — needs 4x A100 GPUs (~$50/hr) and
risks catastrophic forgetting on our small dataset (9.6K examples). LoRA inserts
small trainable matrices (~33M new params, <0.5% of total) into attention and MLP
layers. Original weights stay frozen. Result: trains on a single T4 in ~45-60 min,
adapter weighs ~33MB instead of ~14GB, and the model retains its general language
ability while learning our structured credit output format.

**Phase 2.1 baseline (0% parse rate):**
The base Qwen 2.5-7B writes free-form analyst paragraphs — zero format compliance.
It understands credit concepts (NPAs, downgrades, rating agencies) from pre-training,
but has never seen our `CREDIT_RELEVANT: / DIRECTION: / END` structured format.
This training teaches the format. Any parse rate >80% is a success.

**Time estimate:** ~45-60 min training on T4, ~20-30 min on A100. Eval adds ~15-30 min.

**Data location:** `drive/MyDrive/india-credit-signals/data/processed/`
**Model output:** `drive/MyDrive/india-credit-signals/data/models/qwen-credit-lora/`

---
## Cell 1: Setup — GPU, Packages, Drive

In [ ]:
# ============================================================
# 🎓 Step 1a: Verify GPU allocation
# T4 (15GB VRAM) is fine for QLoRA training of 7B models.
# A100 (40GB) will be faster but not required.
# With 4-bit quantized base + fp16 LoRA adapters, peak VRAM ~10-12GB.
# ============================================================
!nvidia-smi

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name} ({gpu_mem:.1f} GB)")
    # 🎓 bf16 is better than fp16 for training (wider dynamic range, fewer inf/nan)
    # but only Ampere+ GPUs (A100, A10G) support it. T4 is Turing → fp16 only.
    BF16_OK = torch.cuda.get_device_capability()[0] >= 8
    print(f"bf16 support: {BF16_OK} ({'Ampere+' if BF16_OK else 'Turing — using fp16'})")
else:
    raise RuntimeError("No GPU! Go to Runtime → Change runtime type → select GPU")

In [ ]:
# ============================================================
# 🎓 Step 1b: Install dependencies
# pip (not uv) because Colab's pre-built environment conflicts with uv.
#
# New packages vs Phase 2.1:
#   peft: HuggingFace's LoRA/adapter library (creates the small trainable matrices)
#   trl: Transformer Reinforcement Learning library — its SFTTrainer handles:
#        - Chat template formatting (Qwen's <|im_start|>/<|im_end|> tokens)
#        - Prompt masking (loss only on assistant response, not instruction/input)
#        - Mixed precision training (fp16/bf16 automatically)
#   datasets: HuggingFace datasets for data loading with SFTTrainer
# ============================================================
!pip install -q transformers>=4.48.0 accelerate>=1.0.0 bitsandbytes>=0.45.0 \
    peft>=0.14.0 trl>=0.15.0 datasets>=3.0.0 tqdm

import transformers, accelerate, bitsandbytes, peft, trl, datasets as ds_lib
print(f"transformers: {transformers.__version__}")
print(f"accelerate:   {accelerate.__version__}")
print(f"bitsandbytes: {bitsandbytes.__version__}")
print(f"peft:         {peft.__version__}")
print(f"trl:          {trl.__version__}")
print(f"datasets:     {ds_lib.__version__}")
print("✅ Dependencies installed")

In [ ]:
# ============================================================
# 🎓 Step 1c: Mount Google Drive and verify data files
# Drive is the bridge: local Mac → Drive (upload) → Colab (train)
# Trained adapters save back to Drive → download to local for git
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

import os
DATA_DIR = "/content/drive/MyDrive/india-credit-signals/data/processed"
MODEL_DIR = "/content/drive/MyDrive/india-credit-signals/data/models/qwen-credit-lora"
assert os.path.isdir(DATA_DIR), f"Data directory not found: {DATA_DIR}\nUpload data to Google Drive first."

# Create model output directory
os.makedirs(MODEL_DIR, exist_ok=True)

# Verify all required files
required_files = ["train.jsonl", "val.jsonl", "test.jsonl", "entity_holdout.jsonl"]
for fname in required_files:
    fpath = os.path.join(DATA_DIR, fname)
    if os.path.exists(fpath):
        with open(fpath) as f:
            n = sum(1 for _ in f)
        print(f"  ✅ {fname}: {n:,d} examples")
    else:
        raise FileNotFoundError(f"  ❌ {fname}: NOT FOUND at {fpath}")

print(f"\n✅ Data: {DATA_DIR}")
print(f"✅ Model output: {MODEL_DIR}")

---
## Cell 2: Load Base Model in 4-bit QLoRA Configuration

┌──────────────────────────────────────────────────────────────┐
│ 🎓 CONCEPT: QLoRA (Quantized LoRA)                          │
│                                                              │
│ QLoRA combines two tricks:                                   │
│ 1. Base model weights compressed to 4-bit (NF4 format)      │
│    → reduces 14GB model to ~5GB in VRAM                     │
│ 2. LoRA adapter weights stay in full precision (fp16/bf16)  │
│    → only these ~33M params get gradient updates            │
│                                                              │
│ During forward pass: 4-bit weights get dequantized to fp16  │
│ on the fly. During backward pass: only LoRA gradients flow. │
│ Base weights never update.                                   │
│                                                              │
│ Result: train a 7B model on a single T4 (15GB VRAM) for     │
│ ~$0 on Colab Pro. Without QLoRA you'd need ≥40GB VRAM.     │
│                                                              │
│ Paper: https://arxiv.org/abs/2305.14314                     │
└──────────────────────────────────────────────────────────────┘

In [ ]:
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

# 🎓 Same BitsAndBytes config as Phase 2.1 — 4-bit NF4 with double quantization.
# NF4 maps the 16 possible 4-bit values to the most common neural network weight
# values (which follow a normal distribution), minimizing quantization error.
# double_quant: quantizes the quantization scale factors themselves → saves ~0.4GB.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if BF16_OK else torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading {MODEL_NAME} in 4-bit...")
t0 = time.time()

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

load_time = time.time() - t0
param_count = sum(p.numel() for p in model.parameters())
gpu_mem_used = torch.cuda.memory_allocated() / 1e9

print(f"✅ Loaded in {load_time:.1f}s")
print(f"   Parameters: {param_count / 1e9:.2f}B")
print(f"   GPU memory used: {gpu_mem_used:.1f} GB")
print(f"   Quantization: 4-bit NF4 + double quant")

---
## Cell 3: Tokenizer Configuration

🎓 **Qwen tokenizer gotchas — three things that MUST be set correctly:**

1. **`pad_token ≠ eos_token`**: If pad and eos are the same token ID, the loss
   function can't distinguish "this is padding (ignore)" from "this is the real
   end of the response (learn to stop here)". Result: model either never stops
   generating or stops randomly mid-output.

2. **`padding_side="right"`**: For causal LMs, pad tokens go at the END (right)
   of the sequence, not the beginning. Left-padding shifts all positions and
   confuses the positional embeddings.

3. **No `bos_token` override**: Qwen 2.5 doesn't use a beginning-of-sequence
   token. If you set one, you get a mysterious duplicate token at the start
   of every generation ("double-bot" artifact).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 🎓 Qwen 2.5 special tokens:
#   <|endoftext|> (ID 151643) — general end-of-text marker
#   <|im_end|> (ID 151645) — end of a chat turn (instruction/user/assistant)
#   <|im_start|> (ID 151644) — start of a chat turn
#
# We set pad_token to <|endoftext|> and keep eos_token as <|im_end|>.
# This way: padding is masked by pad_token_id, and the model learns to
# stop generating when it outputs <|im_end|> (the real end signal).
tokenizer.pad_token = "<|endoftext|>"       # ID 151643 — used for padding
tokenizer.pad_token_id = 151643
# eos_token stays as <|im_end|> (ID 151645) — Qwen's default
tokenizer.padding_side = "right"              # 🎓 Causal LMs need right-padding

print(f"pad_token: {tokenizer.pad_token} (ID {tokenizer.pad_token_id})")
print(f"eos_token: {tokenizer.eos_token} (ID {tokenizer.eos_token_id})")
print(f"padding_side: {tokenizer.padding_side}")
assert tokenizer.pad_token_id != tokenizer.eos_token_id, \
    "pad_token must differ from eos_token to prevent loss masking corruption!"
print("✅ Tokenizer configured (pad ≠ eos verified)")

---
## Cell 4: Load & Format Training Data

🎓 **Data conversion: instruction/input/output → messages format**

Our training JSONL has three fields per example:
```json
{"instruction": "Assess whether...", "input": "Entity: DHFL\nDate: ...", "output": "CREDIT_RELEVANT: Yes\n..."}
```

SFTTrainer expects a `messages` list with `system`/`user`/`assistant` roles:
```json
{"messages": [
    {"role": "system", "content": "Assess whether..."},
    {"role": "user", "content": "Entity: DHFL\nDate: ..."},
    {"role": "assistant", "content": "CREDIT_RELEVANT: Yes\n..."}
]}
```

SFTTrainer then applies Qwen's chat template (`<|im_start|>system\n...\n<|im_end|>\n...`)
and uses `assistant_only_loss=True` to mask the instruction+input tokens from the loss.
The model only learns to predict the assistant's response — not to parrot back the article.

In [ ]:
import json
from datasets import Dataset

def load_jsonl(path):
    """Load JSONL file into list of dicts."""
    records = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                records.append(json.loads(line))
    return records

def to_messages(example):
    """Convert instruction/input/output → messages format for SFTTrainer."""
    return {
        "messages": [
            {"role": "system", "content": example["instruction"]},
            {"role": "user", "content": example["input"]},
            {"role": "assistant", "content": example["output"]},
        ]
    }

# Load raw JSONL
train_raw = load_jsonl(os.path.join(DATA_DIR, "train.jsonl"))
val_raw = load_jsonl(os.path.join(DATA_DIR, "val.jsonl"))

# Convert to messages format
train_messages = [to_messages(ex) for ex in train_raw]
val_messages = [to_messages(ex) for ex in val_raw]

# Create HuggingFace Datasets
train_dataset = Dataset.from_list(train_messages)
val_dataset = Dataset.from_list(val_messages)

# 🎓 Show training data statistics before training (CLAUDE.md rule: never train
# without showing data stats first)
cr_yes_train = sum(1 for ex in train_raw if ex["output"].startswith("CREDIT_RELEVANT: Yes"))
cr_no_train = len(train_raw) - cr_yes_train
det_train = sum(1 for ex in train_raw if "DIRECTION: Deterioration" in ex["output"])
imp_train = sum(1 for ex in train_raw if "DIRECTION: Improvement" in ex["output"])

print("┌─────────────────────────────────────────────────┐")
print("│ Training Data Statistics                         │")
print("├─────────────────────────────────────────────────┤")
print(f"│ Train examples:    {len(train_raw):>6,d}                      │")
print(f"│ Val examples:      {len(val_raw):>6,d}                      │")
print(f"│ Credit-relevant:   {cr_yes_train:>6,d}  ({cr_yes_train/len(train_raw)*100:.1f}%)            │")
print(f"│ Not relevant:      {cr_no_train:>6,d}  ({cr_no_train/len(train_raw)*100:.1f}%)            │")
print(f"│ Deterioration:     {det_train:>6,d}  ({det_train/len(train_raw)*100:.1f}%)            │")
print(f"│ Improvement:       {imp_train:>6,d}  ({imp_train/len(train_raw)*100:.1f}%)            │")
print("└─────────────────────────────────────────────────┘")

# Show one formatted example
print("\nSample messages format (first example):")
for msg in train_messages[0]["messages"]:
    role = msg["role"]
    content = msg["content"][:120] + "..." if len(msg["content"]) > 120 else msg["content"]
    print(f"  [{role:>9s}] {content}")

# 🎓 Check token lengths to set max_seq_length
# We need to know the longest example to set truncation correctly.
# SFTTrainer will apply the chat template, so we measure AFTER template application.
sample_lengths = []
for ex in train_messages[:200]:  # Check first 200 for speed
    text = tokenizer.apply_chat_template(ex["messages"], tokenize=False)
    tokens = tokenizer(text, return_tensors="pt")
    sample_lengths.append(tokens["input_ids"].shape[1])

print(f"\nToken length stats (first 200 examples):")
print(f"  Min: {min(sample_lengths)}, Max: {max(sample_lengths)}, ")
print(f"  Mean: {sum(sample_lengths)/len(sample_lengths):.0f}, ")
print(f"  P95: {sorted(sample_lengths)[int(0.95*len(sample_lengths))]}, ")
print(f"  P99: {sorted(sample_lengths)[int(0.99*len(sample_lengths))]}")
print(f"\n✅ Data loaded and formatted: {len(train_dataset):,d} train, {len(val_dataset):,d} val")

---
## Cell 5: LoRA Configuration

┌──────────────────────────────────────────────────────────────┐
│ 🎓 CONCEPT: LoRA (Low-Rank Adaptation)                      │
│                                                              │
│ Instead of updating all 7.6B parameters, LoRA inserts small │
│ trainable matrices into selected layers. For each target     │
│ layer W (e.g., q_proj with shape 3584×3584), LoRA adds:    │
│                                                              │
│   W' = W + B × A    where A is (r × 3584) and B is (3584 × r) │
│                                                              │
│ With r=16: each adapter pair is (16×3584) + (3584×16) =     │
│ 114,688 params instead of 12.8M. Across 5 target modules   │
│ × 28 layers = ~33M new params total (<0.5% of 7.6B).       │
│                                                              │
│ alpha=32 scales the adapter output: effective lr per layer  │
│ is alpha/r = 32/16 = 2x. Higher alpha → more adapter       │
│ influence. We use 2x because our task (format learning) is  │
│ relatively easy — we don't want to overpower base knowledge.│
│                                                              │
│ Think of it as: teaching a fluent English speaker credit-   │
│ specific jargon. You don't retrain their entire language.   │
│                                                              │
│ Paper: https://arxiv.org/abs/2106.09685                     │
└──────────────────────────────────────────────────────────────┘

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

# 📐 DECISION: LoRA target modules — 5 of 7 available in Qwen 2.5
#   Qwen 2.5 transformer layers have:
#     Attention: q_proj, k_proj, v_proj, o_proj
#     MLP: gate_proj, up_proj, down_proj
#
#   We target: q_proj, v_proj (attention patterns) + gate_proj, up_proj, down_proj (MLP)
#   We skip: k_proj, o_proj (attention)
#
#   PRO: MLP layers control vocabulary/output distribution — critical for learning
#        our structured format (CREDIT_RELEVANT: Yes/No, specific vocab words)
#   PRO: q_proj + v_proj = standard LoRA recipe, proven on many models
#   CON: More target modules = more params = slightly higher VRAM
#   RISK: If VRAM overflows, drop MLP modules first (attention is more important)

lora_config = LoraConfig(
    r=16,                              # 🎓 Rank 16: ~33M trainable params
    lora_alpha=32,                     # 🎓 alpha/r = 2x scaling factor
    lora_dropout=0.05,                 # 🎓 Light dropout to prevent overfitting on 9.6K examples
    target_modules=[
        "q_proj", "v_proj",            # Attention: what to attend to
        "gate_proj", "up_proj", "down_proj",  # MLP: output vocabulary/format
    ],
    task_type=TaskType.CAUSAL_LM,
    bias="none",                       # 🎓 Standard: don't train bias terms
)

# Apply LoRA to the quantized base model
model = get_peft_model(model, lora_config)

# Print trainable parameter summary
model.print_trainable_parameters()

# Verify VRAM after LoRA application
gpu_mem_after = torch.cuda.memory_allocated() / 1e9
print(f"\nGPU memory after LoRA: {gpu_mem_after:.1f} GB")
print(f"Headroom for training: {gpu_mem - gpu_mem_after:.1f} GB")
print("✅ LoRA adapters applied")

---
## Cell 6: Training Configuration

📐 **DECISION: Using TRL's SFTTrainer instead of a custom training loop**
- PRO: Handles Qwen chat template application automatically
- PRO: `assistant_only_loss=True` → loss computed only on assistant tokens (our output)
  The model doesn't waste gradient updates learning to reproduce the instruction/article
- PRO: Integrates with HuggingFace's Trainer (checkpointing, logging, eval)
- CON: Less control than a custom loop (fine for our use case)
- RISK: TRL versions can have breaking changes — we pin ≥0.15.0

📐 **DECISION: lr=5e-4 (not 2e-4 as originally planned)**
- QLoRA typically needs higher lr than full fine-tuning because gradients pass
  through quantized weights with reduced precision
- 5e-4 is the QLoRA paper's recommended starting point
- With cosine schedule + 100 warmup steps, the effective lr ramps up gently

In [ ]:
from trl import SFTConfig, SFTTrainer

# 🎓 Training hyperparameters — annotated with WHY for each choice
training_args = SFTConfig(
    # --- Output ---
    output_dir="/content/lora_checkpoints",  # 🎓 Local Colab disk (fast), final model goes to Drive

    # --- Training schedule ---
    num_train_epochs=3,                      # 🎓 3 passes over 9.6K examples. More epochs risk
                                             #     overfitting on our small dataset.
    per_device_train_batch_size=4,           # 🎓 4 examples per GPU step
    gradient_accumulation_steps=4,           # 🎓 Effective batch = 4 × 4 = 16. Larger effective
                                             #     batch → smoother gradients → stable training.
                                             #     Tradeoff: fewer weight updates per epoch.

    # --- Learning rate ---
    learning_rate=5e-4,                      # 🎓 QLoRA sweet spot. Higher than standard fine-tune
                                             #     (2e-4) because quantized gradients need more push.
    lr_scheduler_type="cosine",              # 🎓 Cosine decay: high lr early (learn fast) → low lr
                                             #     late (fine-tune carefully). Better than linear decay.
    warmup_steps=100,                        # 🎓 First 100 steps: lr ramps from 0 → 5e-4.
                                             #     Prevents early gradient explosions on fresh LoRA.

    # --- Precision ---
    bf16=BF16_OK,                            # 🎓 Use bf16 on Ampere+ (A100), fp16 on Turing (T4)
    fp16=not BF16_OK,

    # --- Evaluation ---
    eval_strategy="steps",
    eval_steps=500,                          # 🎓 Eval every 500 steps (~1/3 epoch). Catches
                                             #     overfitting early without slowing training.
    per_device_eval_batch_size=4,

    # --- Checkpointing ---
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,                      # 🎓 Keep only 3 best checkpoints (disk space)
    load_best_model_at_end=True,             # 🎓 After training, roll back to best val loss
    metric_for_best_model="eval_loss",
    greater_is_better=False,                 # 🎓 Lower loss = better

    # --- Logging ---
    logging_steps=50,                        # 🎓 Log every 50 steps for training curve visibility
    report_to="none",                        # 🎓 No W&B/TensorBoard — keep it simple for now

    # --- SFT-specific ---
    # NOTE: max_seq_length is NOT a SFTConfig param — it goes in SFTTrainer directly.
    dataset_text_field=None,                 # 🎓 We use messages format, not raw text

    # --- Memory optimization ---
    gradient_checkpointing=True,             # 🎓 Trades ~30% speed for ~40% VRAM savings.
                                             #     Recomputes activations during backward pass
                                             #     instead of storing them all. Essential for T4.
    optim="adamw_torch_fused",               # 🎓 Fused AdamW: single kernel for param update.
                                             #     Faster + less memory than separate operations.
)

# 🎓 Enable gradient checkpointing on the model
# This must be called before creating the trainer, and requires
# use_reentrant=False for compatibility with LoRA.
model.gradient_checkpointing_enable(
    gradient_checkpointing_kwargs={"use_reentrant": False}
)

# Calculate training steps
total_steps = (len(train_dataset) // (training_args.per_device_train_batch_size
               * training_args.gradient_accumulation_steps)) * training_args.num_train_epochs
print(f"\n📐 Training plan:")
print(f"   Examples: {len(train_dataset):,d}")
print(f"   Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Steps per epoch: {len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")
print(f"   Total steps: ~{total_steps:,d}")
print(f"   Eval every: {training_args.eval_steps} steps")
print(f"   Max seq length: 2048 tokens")
print(f"   Precision: {'bf16' if BF16_OK else 'fp16'}")
print(f"   Gradient checkpointing: ON")
print(f"✅ Training config ready")

---
## Cell 7: Train!

🎓 **What to watch during training:**
- `train_loss` should decrease steadily across steps
- `eval_loss` should decrease then flatten (if it starts INCREASING, that's overfitting)
- If eval_loss increases for 2+ eval points → training has gone too far
  (`load_best_model_at_end=True` handles this automatically)
- Total time: ~45-60 min on T4, ~20-30 min on A100

In [ ]:
# 🎓 SFTTrainer wraps HuggingFace Trainer with SFT-specific features:
# - Applies Qwen's chat template to messages automatically
# - assistant_only_loss: only computes loss on assistant (output) tokens
#   The model learns to GENERATE our structured format, not to REPEAT
#   the instruction and article back. Without this, ~80% of gradient
#   updates would be wasted on the long input text.

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    max_seq_length=2048,                     # 🎓 Covers all our examples (longest ~1,200 tokens).
                                             #     2048 gives headroom without wasting memory.
                                             #     This param belongs on SFTTrainer, NOT SFTConfig.
)

print(f"Starting training...")
print(f"  Train: {len(train_dataset):,d} examples")
print(f"  Val: {len(val_dataset):,d} examples")
print(f"  Watch eval_loss — if it starts increasing, the model is overfitting.")
print()

train_result = trainer.train()

# Print training summary
print("\n" + "=" * 70)
print("TRAINING COMPLETE")
print("=" * 70)
metrics = train_result.metrics
print(f"  Total steps:      {metrics.get('total_flos', 'N/A')}")
print(f"  Training time:    {metrics.get('train_runtime', 0) / 60:.1f} min")
print(f"  Final train loss: {metrics.get('train_loss', 'N/A'):.4f}")
print(f"  Samples/sec:      {metrics.get('train_samples_per_second', 0):.1f}")
print()

# 🎓 Save only the LoRA adapters (not the full model)
# The adapter is ~33MB. The base model is ~14GB. We save the adapter
# and reload the base model + adapter at inference time.
# Saving to Drive so it persists after Colab session ends.
print(f"Saving adapters to {MODEL_DIR}...")
trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)

# Verify save
adapter_files = os.listdir(MODEL_DIR)
adapter_size = sum(os.path.getsize(os.path.join(MODEL_DIR, f))
                   for f in adapter_files if os.path.isfile(os.path.join(MODEL_DIR, f)))
print(f"  Saved {len(adapter_files)} files ({adapter_size / 1e6:.1f} MB)")
print(f"  Files: {adapter_files}")
print(f"✅ Adapters saved to Drive")

---
## Cell 8: Evaluation — Parse Rate + Field Accuracy on Validation Set

🎓 **This is where we see if training worked.**

Phase 2.1 baseline: **0% parse rate** — base model writes free-form paragraphs.
Target: **>80% parse rate** — model produces our structured format correctly.

We evaluate on 500 validation examples (from 2022-01 to 2023-06).
These are examples the model never saw during training but from a similar time period.
This is a fair test of format learning, not generalization to future data.

In [ ]:
# ============================================================
# STRICT OUTPUT PARSER (copied from src/data/parse_training_output.py)
# 🎓 SOURCE OF TRUTH: src/data/parse_training_output.py
# Also in: src/training/evaluate.py, notebooks/phase2_1_base_model_eval.ipynb
# If you change the parser, update the source file first.
# ============================================================

import re
from dataclasses import dataclass, field as dataclass_field
from collections import Counter, defaultdict

VOCAB = {
    "credit_relevant": {"Yes", "No"},
    "direction": {"Deterioration", "Improvement", "Neutral"},
    "signal_type": {
        "liquidity", "asset_quality", "regulatory", "contagion",
        "governance", "funding", "operational", "other",
    },
    "sector_wide": {"Yes", "No"},
    "confidence": {"Low", "Medium", "High"},
}


@dataclass
class ParsedOutput:
    credit_relevant: bool = False
    direction: str = ""
    signal_type: str = ""
    sector_wide: bool = False
    confidence: str = ""
    reasoning: str = ""
    has_end_token: bool = False
    parse_ok: bool = False
    error_field: str = ""
    error_detail: str = ""


def parse_model_output(text, vocab=None):
    if vocab is None:
        vocab = VOCAB
    result = ParsedOutput()
    text = text.strip()

    if text.endswith("END"):
        result.has_end_token = True
        text = text[:-3].strip()
    else:
        result.error_field = "END"
        result.error_detail = "Missing END token"

    fields = {}
    current_key = None
    current_val = ""
    for line in text.split("\n"):
        line = line.strip()
        if not line:
            continue
        matched = False
        for field_name in ("CREDIT_RELEVANT", "DIRECTION", "SIGNAL_TYPE",
                           "SECTOR_WIDE", "CONFIDENCE", "REASONING"):
            prefix = field_name + ":"
            if line.upper().startswith(prefix.upper()):
                if current_key is not None:
                    fields[current_key] = current_val.strip()
                current_key = field_name
                current_val = line[len(prefix):].strip()
                matched = True
                break
        if not matched and current_key is not None:
            current_val += " " + line
    if current_key is not None:
        fields[current_key] = current_val.strip()

    cr_val = fields.get("CREDIT_RELEVANT", "").strip()
    if not cr_val:
        result.error_field = "CREDIT_RELEVANT"
        result.error_detail = "Missing CREDIT_RELEVANT field"
        return result
    if cr_val not in vocab.get("credit_relevant", {"Yes", "No"}):
        result.error_field = "CREDIT_RELEVANT"
        result.error_detail = f"Invalid value '{cr_val}', expected Yes/No"
        return result
    result.credit_relevant = (cr_val == "Yes")

    if not result.credit_relevant:
        reasoning = fields.get("REASONING", "").strip()
        if not reasoning:
            result.error_field = "REASONING"
            result.error_detail = "Missing REASONING for non-credit-relevant article"
            return result
        result.reasoning = reasoning
        if result.has_end_token:
            result.parse_ok = True
        return result

    for field_name, attr, valid_set in [
        ("DIRECTION", "direction", vocab.get("direction", set())),
        ("SIGNAL_TYPE", "signal_type", vocab.get("signal_type", set())),
        ("SECTOR_WIDE", "sector_wide", vocab.get("sector_wide", {"Yes", "No"})),
        ("CONFIDENCE", "confidence", vocab.get("confidence", set())),
    ]:
        val = fields.get(field_name, "").strip()
        if not val:
            result.error_field = field_name
            result.error_detail = f"Missing {field_name} field for credit-relevant article"
            return result
        if val not in valid_set:
            result.error_field = field_name
            result.error_detail = f"Invalid value '{val}', expected one of: {sorted(valid_set)}"
            return result
        if field_name == "SECTOR_WIDE":
            result.sector_wide = (val == "Yes")
        else:
            setattr(result, attr, val)

    reasoning = fields.get("REASONING", "").strip()
    if not reasoning:
        result.error_field = "REASONING"
        result.error_detail = "Missing REASONING field for credit-relevant article"
        return result
    result.reasoning = reasoning
    if result.has_end_token:
        result.parse_ok = True
    return result


def classify_failure(raw_output, parsed):
    text = raw_output.strip()
    refusal_patterns = [
        r"(?i)as an (ai|language model|assistant)",
        r"(?i)i cannot", r"(?i)i'm not able to",
        r"(?i)i don't have", r"(?i)i apologize",
        r"(?i)sorry,?\s+(?:but\s+)?i",
    ]
    for pattern in refusal_patterns:
        if re.search(pattern, text):
            return "refusal"
    known_fields = {"CREDIT_RELEVANT", "DIRECTION", "SIGNAL_TYPE",
                    "SECTOR_WIDE", "CONFIDENCE", "REASONING"}
    found_fields = set()
    for fn in known_fields:
        if fn + ":" in text.upper():
            found_fields.add(fn)
    if not found_fields:
        return "totally_unstructured"
    if parsed.error_field == "END" and parsed.error_detail == "Missing END token":
        test_parsed = parse_model_output(text + "\nEND")
        if test_parsed.parse_ok:
            return "missing_end"
    if "Invalid value" in parsed.error_detail:
        return "wrong_vocab"
    if "Missing" in parsed.error_detail and parsed.error_field in known_fields:
        if len(found_fields) >= 2:
            return "partial_format"
        return "missing_field"
    lines = text.split("\n")
    non_field_lines = []
    for line in lines:
        ls = line.strip()
        if not ls or ls == "END":
            continue
        is_field = any(ls.upper().startswith(f + ":") for f in known_fields)
        if not is_field:
            non_field_lines.append(ls)
    if non_field_lines and len(found_fields) >= 3:
        return "extra_content"
    return "partial_format"


print("✅ Parser and failure classifier defined")

In [ ]:
import random
from tqdm import tqdm

# ============================================================
# Inference helper — reused across val, test, and holdout eval
# ============================================================

def run_inference(model, tokenizer, examples, max_new_tokens=200, desc="Inference"):
    """Run inference on a list of instruction/input/output dicts.

    Returns list of raw output strings (model-generated text).
    """
    raw_outputs = []
    model.eval()

    for example in tqdm(examples, desc=desc):
        messages = [
            {"role": "system", "content": example["instruction"]},
            {"role": "user", "content": example["input"]},
        ]
        prompt = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True,
        )
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
            )

        new_tokens = outputs[0][inputs["input_ids"].shape[1]:]
        response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
        raw_outputs.append(response)

    return raw_outputs


def evaluate_outputs(raw_outputs, ground_truth, split_name="val"):
    """Parse outputs, compute metrics, print report.

    Returns (parsed_results, summary_dict).
    """
    parsed_results = [parse_model_output(raw) for raw in raw_outputs]
    total = len(parsed_results)
    parsed_ok = sum(1 for p in parsed_results if p.parse_ok)
    failed = total - parsed_ok
    parse_rate = parsed_ok / total if total > 0 else 0.0

    print("=" * 70)
    print(f"EVALUATION RESULTS — {split_name.upper()}")
    print("=" * 70)
    print(f"  Total examples:      {total:>6,d}")
    print(f"  Successfully parsed: {parsed_ok:>6,d}  ({parse_rate:.1%})")
    print(f"  Parse failures:      {failed:>6,d}  ({1 - parse_rate:.1%})")
    print()

    # Failure taxonomy
    failure_counts = Counter()
    failure_examples = defaultdict(list)
    for pred, raw in zip(parsed_results, raw_outputs):
        if not pred.parse_ok:
            bucket = classify_failure(raw, pred)
            failure_counts[bucket] += 1
            if len(failure_examples[bucket]) < 2:
                failure_examples[bucket].append(raw[:200])

    if failure_counts:
        print("FAILURE MODES:")
        for bucket, count in failure_counts.most_common():
            pct = count / failed * 100 if failed > 0 else 0
            print(f"  {bucket:25s}  {count:>5d}  ({pct:5.1f}% of failures)")
            for j, ex in enumerate(failure_examples[bucket], 1):
                print(f"    Ex {j}: {ex[:100]}")
        print()

    # Per-field accuracy (only for successfully parsed)
    field_metrics = {
        "credit_relevant": {"total": 0, "correct": 0},
        "direction": {"total": 0, "correct": 0},
        "signal_type": {"total": 0, "correct": 0},
        "sector_wide": {"total": 0, "correct": 0},
        "confidence": {"total": 0, "correct": 0},
    }

    for pred, gt_dict in zip(parsed_results, ground_truth):
        if not pred.parse_ok:
            continue
        gt = parse_model_output(gt_dict["output"])
        if not gt.parse_ok:
            continue
        field_metrics["credit_relevant"]["total"] += 1
        if pred.credit_relevant == gt.credit_relevant:
            field_metrics["credit_relevant"]["correct"] += 1
        if pred.credit_relevant and gt.credit_relevant:
            for fn, pv, gv in [
                ("direction", pred.direction, gt.direction),
                ("signal_type", pred.signal_type, gt.signal_type),
                ("sector_wide", pred.sector_wide, gt.sector_wide),
                ("confidence", pred.confidence, gt.confidence),
            ]:
                field_metrics[fn]["total"] += 1
                if pv == gv:
                    field_metrics[fn]["correct"] += 1

    if parsed_ok > 0:
        print("PER-FIELD ACCURACY:")
        print(f"  {'Field':20s}  {'Total':>6s}  {'Correct':>7s}  {'Accuracy':>8s}")
        print("-" * 50)
        for fn, counts in field_metrics.items():
            if counts["total"] > 0:
                acc = counts["correct"] / counts["total"]
                print(f"  {fn:20s}  {counts['total']:>6d}  {counts['correct']:>7d}  {acc:>8.1%}")
            else:
                print(f"  {fn:20s}  {'N/A':>6s}  {'N/A':>7s}  {'N/A':>8s}")
    print()

    summary = {
        "parse_rate": parse_rate,
        "total": total,
        "parsed_ok": parsed_ok,
        "failure_counts": dict(failure_counts),
        "field_metrics": field_metrics,
    }
    return parsed_results, summary


# --- Run validation evaluation ---
VAL_SAMPLE_SIZE = 500
rng = random.Random(42)

# Stratified sample from validation set
val_cr_yes = [ex for ex in val_raw if ex["output"].startswith("CREDIT_RELEVANT: Yes")]
val_cr_no = [ex for ex in val_raw if ex["output"].startswith("CREDIT_RELEVANT: No")]
half = VAL_SAMPLE_SIZE // 2
val_sample = rng.sample(val_cr_yes, min(half, len(val_cr_yes))) + \
             rng.sample(val_cr_no, min(VAL_SAMPLE_SIZE - min(half, len(val_cr_yes)), len(val_cr_no)))
rng.shuffle(val_sample)

print(f"Running inference on {len(val_sample)} validation examples...")
val_raw_outputs = run_inference(model, tokenizer, val_sample, desc="Val Inference")

# Save val outputs to Drive
val_output_path = os.path.join(DATA_DIR, "finetuned_val_outputs.jsonl")
with open(val_output_path, "w", encoding="utf-8") as f:
    for ex, raw in zip(val_sample, val_raw_outputs):
        f.write(json.dumps({"expected": ex["output"], "generated": raw}, ensure_ascii=False) + "\n")
print(f"✅ Val outputs saved to {val_output_path}")

val_parsed, val_summary = evaluate_outputs(val_raw_outputs, val_sample, "validation")

# Compare to baseline
print("┌──────────────────────────────────────────────────────────┐")
print(f"│  PARSE RATE: {val_summary['parse_rate']:.1%} (fine-tuned) vs 0.0% (base model)  │")
if val_summary['parse_rate'] >= 0.80:
    print(f"│  ✅ TARGET MET: >80% parse rate                          │")
else:
    print(f"│  ⚠️  Below 80% target — check failure modes above        │")
print("└──────────────────────────────────────────────────────────┘")

---
## Cell 9: Evaluation — Test Set (The Honest Number)

🎓 **Why val and test are different:**
- **Validation (Cell 8):** Model tuning feedback. We used val loss to select the best
  checkpoint during training. So val results are slightly optimistic — the model was
  indirectly "tuned" to this data.
- **Test (this cell):** The honest number. Forward-looking period (2023-07 to 2024-12)
  that was NEVER used during training in any way. This simulates production deployment:
  "how well does the model perform on future articles?"

If test parse rate is much worse than val, the model overfit to training-era patterns.

In [ ]:
# Load test data
test_raw = load_jsonl(os.path.join(DATA_DIR, "test.jsonl"))

# Sample 500 from test set (stratified)
TEST_SAMPLE_SIZE = 500
test_cr_yes = [ex for ex in test_raw if ex["output"].startswith("CREDIT_RELEVANT: Yes")]
test_cr_no = [ex for ex in test_raw if ex["output"].startswith("CREDIT_RELEVANT: No")]
half = TEST_SAMPLE_SIZE // 2
test_sample = rng.sample(test_cr_yes, min(half, len(test_cr_yes))) + \
              rng.sample(test_cr_no, min(TEST_SAMPLE_SIZE - min(half, len(test_cr_yes)), len(test_cr_no)))
rng.shuffle(test_sample)

print(f"Running inference on {len(test_sample)} test examples...")
test_raw_outputs = run_inference(model, tokenizer, test_sample, desc="Test Inference")

# Save test outputs
test_output_path = os.path.join(DATA_DIR, "finetuned_test_outputs.jsonl")
with open(test_output_path, "w", encoding="utf-8") as f:
    for ex, raw in zip(test_sample, test_raw_outputs):
        f.write(json.dumps({"expected": ex["output"], "generated": raw}, ensure_ascii=False) + "\n")
print(f"✅ Test outputs saved to {test_output_path}")

test_parsed, test_summary = evaluate_outputs(test_raw_outputs, test_sample, "test")

# Summary comparison
print("\n┌──────────────────────────────────────────────────────┐")
print(f"│  VAL parse rate:  {val_summary['parse_rate']:>6.1%}                          │")
print(f"│  TEST parse rate: {test_summary['parse_rate']:>6.1%}  ← the honest number     │")
print(f"│  BASE parse rate:  0.0%  (Phase 2.1 baseline)        │")
gap = abs(val_summary['parse_rate'] - test_summary['parse_rate'])
if gap > 0.10:
    print(f"│  ⚠️  Val→Test gap: {gap:.1%} — possible overfitting      │")
else:
    print(f"│  ✅ Val→Test gap: {gap:.1%} — stable generalization      │")
print("└──────────────────────────────────────────────────────┘")

---
## Cell 10: Evaluation — Entity Holdout (The Money Shot)

🎓 **Why this is the most important eval:**

The entity holdout tests whether the model learned **text patterns** or just **entity names**.

These 3 entities were completely removed from training:
- **DHFL** (1,243 articles, 91% deterioration): Did the model learn to detect distress
  signals from NPAs, defaults, and regulatory actions — or did it just memorize "DHFL = bad"?
- **Reliance Capital** (688 articles, 80% deterioration): Different crisis arc (insurance
  sector, not housing). Can the model generalize across crisis types?
- **Cholamandalam** (1,372 articles, 12% deterioration): The false-positive control.
  A healthy NBFC with mostly positive/neutral news. If the model says "Deterioration"
  for Cholamandalam articles, it's just guessing "all NBFCs are risky."

The ideal result: high recall on DHFL/RelCap deterioration + low false positive rate on Cholamandalam.

In [ ]:
# Load full entity holdout set
holdout_raw = load_jsonl(os.path.join(DATA_DIR, "entity_holdout.jsonl"))
print(f"Entity holdout: {len(holdout_raw):,d} examples")

# Extract entity names
def extract_entity(input_text):
    for line in input_text.split("\n"):
        if line.startswith("Entity:"):
            return line[len("Entity:"):].strip()
    return "UNKNOWN"

holdout_entities = [extract_entity(ex["input"]) for ex in holdout_raw]
entity_counts = Counter(holdout_entities)
print(f"Entities: {dict(entity_counts)}")

# Run inference on ALL holdout examples
# This takes ~40-60 min on T4. It's worth the wait — this is the definitive eval.
print(f"\nRunning inference on {len(holdout_raw):,d} holdout examples...")
print(f"Estimated time: ~{len(holdout_raw) * 1.5 / 60:.0f} min on T4")
holdout_raw_outputs = run_inference(model, tokenizer, holdout_raw, desc="Holdout Inference")

# Save holdout outputs
holdout_output_path = os.path.join(DATA_DIR, "finetuned_holdout_outputs.jsonl")
with open(holdout_output_path, "w", encoding="utf-8") as f:
    for ex, raw, entity in zip(holdout_raw, holdout_raw_outputs, holdout_entities):
        f.write(json.dumps({
            "entity": entity,
            "expected": ex["output"],
            "generated": raw,
        }, ensure_ascii=False) + "\n")
print(f"✅ Holdout outputs saved to {holdout_output_path}")

# Overall holdout parse eval
holdout_parsed, holdout_summary = evaluate_outputs(
    holdout_raw_outputs, holdout_raw, "entity holdout (aggregate)"
)

# --- Per-entity evaluation ---
print("\n" + "=" * 70)
print("PER-ENTITY HOLDOUT EVALUATION")
print("=" * 70)

ENTITY_INFO = {
    "DHFL": "Crisis detection (91% det.) — did it learn distress patterns?",
    "Reliance Capital": "Generalization (80% det.) — different crisis type?",
    "Cholamandalam": "False positive control (12% det.) — stable NBFC",
}

for entity in ["DHFL", "Reliance Capital", "Cholamandalam"]:
    indices = [i for i, e in enumerate(holdout_entities) if e == entity]
    if not indices:
        print(f"\n  {entity}: No examples found")
        continue

    e_raw_outs = [holdout_raw_outputs[i] for i in indices]
    e_ground_truth = [holdout_raw[i] for i in indices]
    e_parsed = [holdout_parsed[i] for i in indices]

    total = len(indices)
    parsed_ok = sum(1 for p in e_parsed if p.parse_ok)
    pr = parsed_ok / total if total > 0 else 0.0

    # Per-field accuracy
    cr_correct = 0
    dir_correct = dir_total = 0
    st_correct = st_total = 0
    det_tp = det_fp = det_fn = 0
    imp_tp = imp_fp = imp_fn = 0

    for pred, gt_dict in zip(e_parsed, e_ground_truth):
        if not pred.parse_ok:
            continue
        gt = parse_model_output(gt_dict["output"])
        if not gt.parse_ok:
            continue
        if pred.credit_relevant == gt.credit_relevant:
            cr_correct += 1
        if pred.credit_relevant and gt.credit_relevant:
            dir_total += 1
            if pred.direction == gt.direction:
                dir_correct += 1
            st_total += 1
            if pred.signal_type == gt.signal_type:
                st_correct += 1

        pred_det = pred.credit_relevant and pred.direction == "Deterioration"
        gt_det = gt.credit_relevant and gt.direction == "Deterioration"
        pred_imp = pred.credit_relevant and pred.direction == "Improvement"
        gt_imp = gt.credit_relevant and gt.direction == "Improvement"

        if pred_det and gt_det: det_tp += 1
        elif pred_det and not gt_det: det_fp += 1
        elif not pred_det and gt_det: det_fn += 1
        if pred_imp and gt_imp: imp_tp += 1
        elif pred_imp and not gt_imp: imp_fp += 1
        elif not pred_imp and gt_imp: imp_fn += 1

    info = ENTITY_INFO.get(entity, "")
    print(f"\n  {entity} ({total} articles) — {info}")
    print(f"  {'─' * 60}")
    print(f"    Parse rate:         {pr:>6.1%}  ({parsed_ok}/{total})")
    if parsed_ok > 0:
        print(f"    CR accuracy:        {cr_correct / parsed_ok:>6.1%}")
        print(f"    Direction accuracy:  {dir_correct / dir_total:>6.1%}" if dir_total > 0 else "    Direction accuracy:    N/A")
        print(f"    Signal type acc:    {st_correct / st_total:>6.1%}" if st_total > 0 else "    Signal type acc:      N/A")
        det_prec = det_tp / (det_tp + det_fp) if (det_tp + det_fp) > 0 else 0.0
        det_rec = det_tp / (det_tp + det_fn) if (det_tp + det_fn) > 0 else 0.0
        imp_prec = imp_tp / (imp_tp + imp_fp) if (imp_tp + imp_fp) > 0 else 0.0
        imp_rec = imp_tp / (imp_tp + imp_fn) if (imp_tp + imp_fn) > 0 else 0.0
        print(f"    Det. precision:     {det_prec:>6.1%}  (of predicted det., how many correct)")
        print(f"    Det. recall:        {det_rec:>6.1%}  (of actual det., how many caught)")
        print(f"    Imp. precision:     {imp_prec:>6.1%}")
        print(f"    Imp. recall:        {imp_rec:>6.1%}")

    # Failure modes for this entity
    e_failures = Counter()
    for pred, raw in zip(e_parsed, e_raw_outs):
        if not pred.parse_ok:
            e_failures[classify_failure(raw, pred)] += 1
    if e_failures:
        print(f"    Failure modes: {dict(e_failures)}")

# ============================================================
# FINAL SUMMARY
# ============================================================
print("\n" + "=" * 70)
print("PHASE 2.2 — TRAINING COMPLETE — FINAL SUMMARY")
print("=" * 70)
print(f"")
print(f"  ┌────────────────────────────────────────────────────────────────┐")
print(f"  │  Model:   Qwen 2.5-7B-Instruct + LoRA (r=16, 5 modules)      │")
print(f"  │  Train:   {len(train_raw):>6,d} examples, {training_args.num_train_epochs} epochs                        │")
print(f"  │                                                                │")
print(f"  │  Parse rates:                                                  │")
print(f"  │    Base model (Phase 2.1):   0.0%                              │")
print(f"  │    Validation (post-train): {val_summary['parse_rate']:>5.1%}                              │")
print(f"  │    Test (honest number):    {test_summary['parse_rate']:>5.1%}                              │")
print(f"  │    Entity holdout:          {holdout_summary['parse_rate']:>5.1%}                              │")
print(f"  │                                                                │")
print(f"  │  Adapters saved to: {MODEL_DIR}  │")
print(f"  └────────────────────────────────────────────────────────────────┘")
print()
print(f"  ⏭️  NEXT STEPS:")
print(f"  1. Review per-entity holdout results above")
print(f"  2. If parse rate >80% → model is working, proceed to Phase 2.3 (RLMF)")
print(f"  3. If field accuracy is low → may need more training data or adjusted hyperparams")
print(f"  4. Download adapters from Drive for git commit")

# Clean up GPU memory
del model, tokenizer
torch.cuda.empty_cache()
print("\n✅ GPU memory freed")